In [1]:
# import library
import requests, sqlite3
import pandas as pd
from bs4 import BeautifulSoup
# use this lib to convert the rate from string to int
from word2number import w2n

## Books Table

In [2]:
# make connection with two tables
connection = sqlite3.connect('books.db')
# create table books
cursor = connection.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS books(
                Id Int PRIMARY KEY,
                Title TEXT, 
                Price Float, 
                Rate INT);''')

cursor.execute('''CREATE TABLE IF NOT EXISTS categories(
                Id Int,
                Category TEXT,
                FOREIGN KEY(Id) 
                    REFERENCES books(Id) 
                    ON DELETE CASCADE 
                    ON UPDATE CASCADE
                );''')
# this link realated to each book so can take category from it
url_to_get_category = "https://books.toscrape.com/catalogue/"
book_num = 1
for j in range(1,51):
    # links of this pages from 1 to 50
    url = f"https://books.toscrape.com/catalogue/page-{j}.html"
    # to make a request
    response = requests.get(url)
    # to take snapshot of the website as a text
    snap = BeautifulSoup(response.text, 'html.parser')
    # take the order list that have all books
    res = snap.find_all('ol')
    # take all h3 in res. 
    # here we select[0] because to could find_all in it
    names = res[0].find_all('h3')
    # take all p in res
    # here we select[0] because to could find_all in it
    details = res[0].find_all('p')
    # assign this variable to use it in for loop
    rate = 0
    price = 1
    """
    use loop to loop in each book and get the information from it
    """
    for i in range(len(res[0].find_all('li'))):
        # the name
        n = names[i].find('a').get('title')
        # the price
        p = details[price].get_text().replace('Â£','')
        # the rate
        r = w2n.word_to_num(details[rate].get('class')[1]) # here convert the rate from text to number
        # take all h3 in snap
        get_href = snap.find_all('h3')
        # get the link of book
        link = get_href[i].find('a').get('href')
        # make request with new site to get the category of each book
        response2 = requests.get(url_to_get_category+link)
        # to take snapshot of the website as a text
        snap2 = BeautifulSoup(response2.text, 'html.parser')
        # the category
        category = snap2.find('ul').find_all('li')[2].find('a').get_text()
        # pass values in the databases
        try:
            cursor = cursor.execute("INSERT INTO books (Id,Title,Price,Rate) VALUES (?, ?, ?, ?);", [book_num,n, p, r])
            cursor = cursor.execute("INSERT INTO categories (Id, Category) VALUES (?, ?);", [book_num, category])
        # here exception for any error when enter data for each books
        except Exception as ex:
            print('The Error is', ex)
        # make commit
        connection.commit()
        # increase the rate and price to get the data from next book
        rate = rate + 3
        price = rate + 1
        # to increase the id of each book
        book_num = book_num + 1
    print(f'Scraping page {j} done')
# close the databases
connection.close()

Scraping page 1 done
Scraping page 2 done
Scraping page 3 done
Scraping page 4 done
Scraping page 5 done
Scraping page 6 done
Scraping page 7 done
Scraping page 8 done
Scraping page 9 done
Scraping page 10 done
Scraping page 11 done
Scraping page 12 done
Scraping page 13 done
Scraping page 14 done
Scraping page 15 done
Scraping page 16 done
Scraping page 17 done
Scraping page 18 done
Scraping page 19 done
Scraping page 20 done
Scraping page 21 done
Scraping page 22 done
Scraping page 23 done
Scraping page 24 done
Scraping page 25 done
Scraping page 26 done
Scraping page 27 done
Scraping page 28 done
Scraping page 29 done
Scraping page 30 done
Scraping page 31 done
Scraping page 32 done
Scraping page 33 done
Scraping page 34 done
Scraping page 35 done
Scraping page 36 done
Scraping page 37 done
Scraping page 38 done
Scraping page 39 done
Scraping page 40 done
Scraping page 41 done
Scraping page 42 done
Scraping page 43 done
Scraping page 44 done
Scraping page 45 done
Scraping page 46 do

## books table

In [3]:
connection = sqlite3.connect("books.db")
df = pd.read_sql('''SELECT *
                    FROM books''',connection)
df

,Id,Title,Price,Rate
0,1,A Light in the Attic,51.77,3
1,2,Tipping the Velvet,53.74,1
2,3,Soumission,50.10,1
3,4,Sharp Objects,47.82,4
4,5,Sapiens: A Brief History of Humankind,54.23,5
5,6,The Requiem Red,22.65,1
6,7,The Dirty Little Secrets of Getting Your Dream...,33.34,4
7,8,The Coming Woman: A Novel Based on the Life of...,17.93,3
8,9,The Boys in the Boat: Nine Americans and Their...,22.60,4
9,10,The Black Maria,52.15,1


## categories table

In [4]:
connection = sqlite3.connect("books.db")
df = pd.read_sql('''SELECT *
                    FROM categories''',connection)
df

,Id,Category
0,1,Poetry
1,2,Historical Fiction
2,3,Fiction
3,4,Mystery
4,5,History
5,6,Young Adult
6,7,Business
7,8,Default
8,9,Default
9,10,Poetry


In [5]:
connection.close()